In [1]:
import os
import torchvision
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import cv2 as cv2
from  PIL import Image
from PIL import ImageFile 
import torch
from sklearn import metrics
import torch.nn as nn
package_paths = [
    '../input/timmmaster',
]

import sys


for pth in package_paths:
    sys.path.append(pth)
from torch.optim.lr_scheduler import OneCycleLR
from albumentations.core.composition import Compose,OneOf
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import timm

In [2]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [3]:
def makefilelist(path):
    file_path=[]
    labels=[]
    for i in os.listdir(path):
        for j in os.listdir(path+i):
            file_path.append(path+i+'/'+j)
            labels.append(i)
    return file_path,labels        

In [4]:

test_path='../input/mnist0100/MNIST(0-100)/test/'
test_filepath,test_labels=makefilelist(test_path)        


In [5]:
torch.manual_seed(2022);

In [6]:
class CFG:
    seed = 42
    img_size = 64
    num_classes = 101
    lr = 1e-2
    max_lr = 1e-2
    pct_start = 0.2
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    epochs = 10
    batch_size = 128
    accum = 1
    precision = 16
    n_fold = 4
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset

In [7]:
import albumentations as A
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
#             A.HueSaturationValue(p=0.5),
#             A.OneOf([
#                 A.RandomBrightnessContrast(p=0.5),
#                 A.RandomGamma(p=0.5),
#             ], p=0.5),
#             A.OneOf([
#                 A.Blur(p=0.1),
#                 A.GaussianBlur(p=0.1),
#                 A.MotionBlur(p=0.1),
#             ], p=0.1),
#             A.OneOf([
#                 A.GaussNoise(p=0.1),
#                 A.ISONoise(p=0.1),
#                 A.GridDropout(ratio=0.5, p=0.2),
#                 A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
#             ], p=0.2),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [8]:
class mnistdataset(Dataset):
    def __init__(self,path,labels,aug):
        self.path=path
        self.labels=labels
        self.aug=aug
        #self.resize=resize
    def __len__(self):
        return len(self.path)
        
        
    def __getitem__(self,index):
        
        
        image_path= self.path[index]
        image = cv2.imread(image_path)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        target=int(self.labels[index])
            
            
      
        if self.aug is not None:
            image=self.aug(image=image)['image']
            #image=self.aug(image)
#             image=np.transpose(image,(2,0,1)).astype(np.float32)
            #image=image/255.0
        return  {
            "image":image.float(),
            "target":torch.tensor(target,dtype=torch.long)
        }
        
        
    

In [9]:
test_dataset=mnistdataset(test_filepath,test_labels,aug=get_transform('train'))


In [10]:
test_loader=torch.utils.data.DataLoader(test_dataset,shuffle=True,batch_size=CFG.batch_size)


In [11]:
# def imshow(img):
#     img=img/2+0.5
#     npimg=img.numpy()
#     plt.imshow(np.transpose(npimg,(1,2,0)))
#     #plt.imshow(npimg)
#     plt.show()
    
# dataiter=iter(test_loader)
# images=dataiter.next()
# #print(images['image'][0])
# imshow(images['image'][0])
# image=images['image'][0]
# image=image.unsqueeze(0)
# image=image.to(device)
# print(image.shape)
# pred=model(image)
# print(torch.max(pred,1)[1])


# Model

In [12]:
class mnistmodel(nn.Module):
    """A simple convolutional network to tackle Digit Recognizer competition."""

    def __init__(self):
        super().__init__()

        self.conv0 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn0 = nn.BatchNorm2d(64)
        self.pool0 = nn.MaxPool2d(2)
        self.dropout0 = nn.Dropout2d(p=0.27)

        self.conv1 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.pool1 = nn.MaxPool2d(2)
        self.dropout1 = nn.Dropout2d(p=0.27)

#         self.conv2 = nn.Conv2d(128, 192, 3, padding=1)
#         self.bn2 = nn.BatchNorm2d(192)
#         self.pool2 = nn.MaxPool2d(2)
#         self.dropout2 = nn.Dropout2d(p=0.27)

#         self.global_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=32768, out_features=101)

          
      
        
    def forward(self, x):
        out = self.conv0(x)
        out = self.bn0(out).relu()
        out = self.pool0(out)
        out = self.dropout0(out)

        out = self.conv1(out)
        out = self.bn1(out).relu()
        out = self.pool1(out)
        out = self.dropout1(out)

#         out = self.conv2(out)
#         out = self.bn2(out).relu()
#         out = self.pool2(out)
#         out = self.dropout2(out)

#         out = self.global_pool(out)
        out = out.reshape(-1,32768 )
        out = self.fc(out)
        return out
       

# Load model weights

In [13]:
from torch.optim import Adam,SGD
model=mnistmodel()
model.to(device)
model.load_state_dict(torch.load('../input/nybsys-project-demo/Epoch-7--with--val_loss--0.760'))
                                

<All keys matched successfully>

# Inference

In [14]:
from tqdm import tqdm

loop=tqdm(test_loader,leave=False)
model.eval()
val_acc_list=[]
with torch.no_grad():
    i=0
    for data in loop:
        i+=1
        image=data['image'].to(device)
        target=data['target'].to(device)
        output=model(image)
        _,final_output=torch.max(torch.nn.functional.softmax(output,1),1)
        cor=torch.sum(target==final_output)
        acc=(cor*100)/len(target)
        val_acc_list.append(acc.item())
    print("------Accuracy-----")       
    print(sum(val_acc_list)/i)    
    

------Accuracy-----
82.06705590743053
